In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Komoran
from tqdm.notebook import tqdm

In [2]:
train_data = pd.read_parquet('./data/chosun_1990.parquet')
train_data[:5] # 상위 5개 출력

,Unnamed: 0,publication_date,page_section,title_archaic_hangul,title_archaic,body_archaic_hangul,body_archaic
0,3339676,1990-01-01,정치,전두환(全斗煥)씨 국회(国会)증언 요지,全斗煥씨 国会증언 요지,"전두환(全斗煥)씨 국회(国会)증언 요지\n""계엄군(軍) 강경진압 광주(光州)사태 주...","全斗煥씨 国会증언 요지\n""계엄軍 강경진압 光州사태 주원인""\n「12—12」는 수사..."
1,3339677,1990-01-01,정치,◇기자(記者)회견하는 전(全)씨,◇記者회견하는 全씨,◇기자(記者)회견하는 전(全)씨\n 전두환(全斗煥) 전(前)대통령이 31일 자정직전...,◇記者회견하는 全씨\n 全斗煥 前대통령이 31일 자정직전 기자회견을 통해 미처 증언...
2,3339678,1990-01-01,정치,"4당(党)-재야(在野)의 새로운""헤쳐모여""관건","4党-在野의 새로운""헤쳐모여""관건","4당(党)-재야(在野)의 새로운""헤쳐모여""관건\n청산(清算)이후의 90년정국(政局)...","4党-在野의 새로운""헤쳐모여""관건\n清算이후의 90년政局 기상도 정치부 좌담\n与圈..."
3,3339679,1990-01-01,정치,전두환(全斗煥)씨 국회(国会)증언 요지,全斗煥씨 国会증언 요지,"전두환(全斗煥)씨 국회(国会)증언 요지\n""도청(道庁)앞 발포(発砲) 상황끝난뒤 보...","全斗煥씨 国会증언 요지\n""道庁앞 発砲 상황끝난뒤 보고받아""\n「光州」인식엔 美와 ..."
4,3339680,1990-01-01,정치,"""루마니아 공산당(共産党) 해체""","""루마니아 共産党 해체""","""루마니아 공산당(共産党) 해체""\n【부쿠레슈티=APUPI연합(聯合)】루마니아 공산...","""루마니아 共産党 해체""\n【부쿠레슈티=APUPI聯合】루마니아 공산당원들은 31일 ..."


In [69]:
print(train_data.isnull().values.any())

False


In [3]:
train_data = train_data['body_archaic_hangul']
train_data[:5]

0    전두환(全斗煥)씨 국회(国会)증언 요지\n"계엄군(軍) 강경진압 광주(光州)사태 주...
1    ◇기자(記者)회견하는 전(全)씨\n 전두환(全斗煥) 전(前)대통령이 31일 자정직전...
2    4당(党)-재야(在野)의 새로운"헤쳐모여"관건\n청산(清算)이후의 90년정국(政局)...
3    전두환(全斗煥)씨 국회(国会)증언 요지\n"도청(道庁)앞 발포(発砲) 상황끝난뒤 보...
4    "루마니아 공산당(共産党) 해체"\n【부쿠레슈티=APUPI연합(聯合)】루마니아 공산...
Name: body_archaic_hangul, dtype: object

In [4]:
from hanja.impl import is_hanja as is_hanja
from hanja.hangul import is_hangul as is_hangul

def recognize_hanja(string):
    hanja_cnt = 0
    hanja_reversed = ''
    hangul_cnt = 0
    hangul_reversed = ''
    hanja_removed_reversed = ''
    hanja_words = {}   
    
    for s in string[::-1]:
        if is_hanja(s):
            hanja_cnt += 1
            hanja_reversed += s
        elif hanja_cnt > 0 and is_hangul(s):
            hangul_cnt += 1
            hangul_reversed += s
            hanja_removed_reversed += s
        
            if hanja_cnt != 0 and hangul_cnt != 0 and hangul_cnt == hanja_cnt:
                hanja_words[hangul_reversed[::-1]] = hanja_reversed[::-1]
                hanja_cnt = 0
                hanja_reversed = ''
                hangul_cnt = 0
                hangul_reversed = ''
                continue
        else:
            hanja_removed_reversed += s
    
    return hanja_words, hanja_removed_reversed[::-1]

In [5]:
STOPWORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', ',', '.', 'ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
KOM = Komoran()

def hanja_tokenize_string(string):
    hanja_words, hanja_removed = recognize_hanja(string)
    hanja_removed = hanja_removed.replace('()', '')
    tokenized_string = KOM.morphs(hanja_removed)
    r = []
    for token in tokenized_string:
        if token in hanja_words:
            r.append(hanja_words[token])
            continue
        if token in STOPWORDS:
            continue
        r.append(token)
    return r
    

In [62]:
hanja_words, hanja_removed = recognize_hanja('젊은이반대(反對)의 힘 용납할때 화해점(和解點) 발견')
hanja_removed = hanja_removed.replace('()', '')
hanja_removed

'젊은이반대의 힘 용납할때 화해점 발견'

In [60]:
hanja_tokenize_string('젊은이반대(反對)의 힘 용납할때 화해점(和解點) 발견')

['젊은이', '反對', '힘', '용납', '하', '때', '화해', '점', '발견']

In [6]:
r = []
for i in tqdm(range(len(train_data))):
    try:
        r.append(hanja_tokenize_string(train_data[i]))
    except Exception as e:
        print(e)
        print(train_data[i])

  0%|          | 0/66214 [00:00<?, ?it/s]

'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte
말많은「신당(新党) 신접살이」
당명(党名)—당사(党舍)문제등 연이은 마찰
「민자(民自)」싸고 고수(固守)—현상공모 맞서
당사(党舍)내정「금융연(研)」선 노조(勞組)반발
 오는 15일로 예정된 민자당(民自党)(가칭)의 중앙선관위등록을 앞두고 민정 민주 공화3정파(政派)의 통합추진위는 창당준비작업에 박차를 가하고 있다.그러나 순탄하지만은 않다.주요 당직인선(人選),지구당조직책선정등「사람」문제와는달리 3당간에 비교적 의견통일이 쉬울것으로 예상됐던「제도」문제마저도 곳곳에서걸리고 있다.
 3당(党)대표들이「공동대표제」로 신당(新党)지도체제에 합의,제도의 기본틀은 일단 마련됐으나 다른의견이 그치지 않는 당명(党名),뒤늦게 이견(異見)이 제기된 정강정책등이실무진들의마음을 바쁘게하고 있는것.
「민주전진당(民主前進党)」등 백출(百出)
 〇…김영삼민주당(金泳三民主党)총재의제안에 따라 일찌감치 민주자유당(民主自由党)으로 결정된 당명(党名)은 그약칭에 대해 처음부터『어감이 좋지 않다』『일본(日本) 자민당(自民党)복사판 같다』는 등의 부정적여론이 돌더니 최근들어「대한민주당(大韓民主党)」,「자유통일당(自由統一党)」,「우리국민당(国民党)」,「통일한국당(統一韓国党)」,「민주자유연합(民主自由聯合)」등 구체적 대안까지 나오는 지경이다.
 지난5일 민정당고문들이참석한 청와대만찬에서도 개명(改名)문제가 거론됐으며 6일의민정당 중집위(中執委)에서는 이종찬정석모(李鍾賛鄭石謨)의원등이『당명이 모방성이강하고 희화화(戱画化)되고있다』면서 현상공모(公募) 방안까지거론했다.
 윤길중(尹吉重)고문도『시간이 촉박해 국민의 의견을 듣기 어려우면 의원들에게만이라도 좋은 의견을 듣자』고 제의했는데 회의결과를 전한 박희태(朴熺太)대변인은 기자들에게까지「묘안(妙案)」을 호소.
 이에대해 공화당측은『당가,당기,당심벌마크는 몰라도 당명을 공모할 시간은없다』며「민자당(民自党)」고수입장을 

In [19]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = r, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [20]:
print(model.wv.most_similar("앞날"))

[('장래', 0.6852638721466064), ('미래', 0.6622290015220642), ('작금', 0.6565095782279968), ('危機', 0.6468039155006409), ('향방', 0.6320191621780396), ('위기감', 0.6282010078430176), ('정치판', 0.615230917930603), ('난국', 0.6021440029144287), ('국면', 0.5988582372665405), ('활력', 0.5984665751457214)]


# Todo
형태소 분석 후에 저장